In [1]:
import pandas as pd

# 파일 경로
file1 = '/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_file.csv'
file2 = '/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_auto_tune.csv'
output_path = '/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_all.csv'

# CSV 파일 읽기
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# 데이터 합치기
df_merged = pd.concat([df1, df2], ignore_index=True)

# 저장
df_merged.to_csv(output_path, index=False)
print(f"✅ 파일이 성공적으로 합쳐졌습니다: {output_path}")


✅ 파일이 성공적으로 합쳐졌습니다: /home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_all.csv


### subset 나누기

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 🔍 경로 및 데이터 불러오기
meta_path = "/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta.csv"
df = pd.read_csv(meta_path)

# 📌 분할 컬럼 추가
df['Split'] = None

# ✅ 1. Bonafide 분할
# 1-1. CommonVoice는 eval
df.loc[(df['label2'] == 'bonafide') & (df['group'] == 'TIMIT'), 'Split'] = 'eval'

# 1-2. VCTK & LibriSpeech → train/dev 분할
real_train_dev = df[(df['label2'] == 'bonafide') & (df['group'].isin(['VCTK', 'LibriSpeech']))]
train_idx, dev_idx = train_test_split(real_train_dev.index, test_size=0.3, random_state=42, shuffle=True)
df.loc[train_idx, 'Split'] = 'train'
df.loc[dev_idx, 'Split'] = 'dev'

# ✅ 2. Spoof 분할
spoof_df = df[df['label2'] == 'spoof']
for group_name in spoof_df['group'].unique():
    group_data = spoof_df[spoof_df['group'] == group_name]
    
    # 50% → eval
    group_eval_idx = group_data.sample(frac=0.5, random_state=42).index
    df.loc[group_eval_idx, 'Split'] = 'eval'
    
    # 나머지 → train/dev
    remaining = group_data.drop(index=group_eval_idx)
    group_train_idx, group_dev_idx = train_test_split(remaining.index, test_size=0.3, random_state=42)
    df.loc[group_train_idx, 'Split'] = 'train'
    df.loc[group_dev_idx, 'Split'] = 'dev'

# ✅ 검증
print(df['Split'].value_counts())

# 💾 저장
output_path = "/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_subset.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ 데이터셋 분할 완료: {output_path}")


train    270269
eval     152900
dev      115831
Name: Split, dtype: int64

✅ 데이터셋 분할 완료: /home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_subset.csv


### Protocol 파일 만들기

In [1]:
import pandas as pd

# 데이터 로드
df = pd.read_csv('/home/woongjae/multi-feature_fusion/Datasets/meta_subset.csv')

# 프로토콜 텍스트 만들기 함수
def make_protocol_multi(df, subset_list, protocol_path):
    # 여러 subset 필터링 (예: train, dev)
    proto_df = df[df['Subset'].str.lower().isin(subset_list)]
    with open(protocol_path, 'w') as f:
        for _, row in proto_df.iterrows():
            # Train/dev 모두 'Train'으로 통일
            f.write(f"{row['file_path']}\tTrain\t{row['label1']}\n")

# 사용 예시: train/dev 합친 파일 만들기
make_protocol_multi(df, ['train', 'dev'], 'protocol_train_dev.txt')

# eval은 기존대로 (참고)
make_protocol_multi(df, ['eval'], 'protocol_eval.txt')


### protocol 파일 경로 바꾸기

In [4]:
# 프로토콜 파일을 한 줄씩 읽어서 경로 일괄 치환
# input_path = "protocol_train_dev.txt"   # 원본 파일
# output_path = "protocol_train_dev_fixed.txt"  # 저장할 파일
input_path = "protocol_eval.txt"   # 원본 파일
output_path = "protocol_eval_f.txt"  # 저장할 파일
old_prefix = "home/woongjae/noise-tracing/muti-feature_fusion"
new_prefix = "home/woongjae/multi-feature_fusion"

with open(input_path, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        # 경로 치환
        new_line = line.replace(old_prefix, new_prefix)
        fout.write(new_line)

print(f"✅ 경로 치환 완료! 결과 저장 위치: {output_path}")


✅ 경로 치환 완료! 결과 저장 위치: protocol_eval_f.txt


### protocol 파일 레이블 변경

In [6]:
# protocol 파일에서 label1 치환(수정)
# input_path = "protocol_train_dev.txt"    # 기존 프로토콜 파일명
# output_path = "protocol_train_dev_fixed.txt"   # 수정된 파일명

input_path = "protocol_eval.txt"    # 기존 프로토콜 파일명
output_path = "protocol_eval_f.txt"   # 수정된 파일명

# 치환 매핑 딕셔너리
replace_dict = {
    "high_pass_filter": "band_pass_filter",
    "low_pass_filter": "band_pass_filter",
    "pitch_shift": "manipulation",
    "time_stretch": "manipulation"
}

with open(input_path, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        items = line.strip().split('\t')
        if len(items) != 3:
            fout.write(line)  # 포맷 이상 줄은 그대로 출력
            continue
        # label1 치환
        label1 = replace_dict.get(items[2], items[2])
        fout.write(f"{items[0]}\t{items[1]}\t{label1}\n")

print(f"✅ 프로토콜 파일 라벨 치환 완료! 저장 위치: {output_path}")


✅ 프로토콜 파일 라벨 치환 완료! 저장 위치: protocol_eval_f.txt
